<a href="https://colab.research.google.com/github/gamma86/github-final-project/blob/main/Clusterizaci%C3%B3n_Clasificaci%C3%B3n_de_papers_con_base_en_sus_abstracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

NSF Research Awards Abstracts

En este proyecto se analizará parte de la información contenida en un conjunto de abstracts de artículos, los cuales pueden obtenerse de esta liga https://www.nsf.gov/awardsearch/download?DownloadFileName=2020&All=true
    
El objetivo es ubicar los abstracts dentro de cierto tema, es decir, agrupar/clasificar los abstracts con base en su contenido.

El problema será abordado utilizando tanto herramientas de IA generativa, como de ML tradicional.

# Importamos librerías

In [20]:
import requests
import zipfile
import os
from lxml import etree

import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.language_models import TextGenerationModel

from sklearn.cluster import KMeans

import math
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from typing import Generator, List, Optional, Tuple
import functools
import time
from concurrent.futures import ThreadPoolExecutor

import pickle

### Credenciales

In [13]:
# Autenticación
from google.colab import auth as google_auth
google_auth.authenticate_user()

# Extracción de datos

In [21]:
# Descargamos la carpeta
url = "https://www.nsf.gov/awardsearch/download?DownloadFileName=2020&All=true"
response = requests.get(url)
with open("awardsearch_2020.zip", "wb") as zip_file:
    zip_file.write(response.content)
with zipfile.ZipFile("awardsearch_2020.zip", "r") as zip_ref:
    zip_ref.extractall("descomprimido")
    nombres_archivos = os.listdir("descomprimido")

In [22]:
# Obtenemos las rutas de los archivos xml
folder_path = './descomprimido'
file_names = os.listdir(folder_path)
rutas = []
rutas = [folder_path + "/" + file for file in file_names]

# Por ahora solo trabajaremos con el título y el abstract de los archivos xml
titulos_abstracts = []

for file in rutas:
    try:
        with open(file) as f:
            info = etree.parse(f)
            titulo = info.findtext('.//AwardTitle')
            abstract = info.findtext('.//AbstractNarration')
            titulos_abstracts.append((titulo, abstract))
    except Exception as e:
        print(f"Error al procesar el archivo {file}: {e}")

In [23]:
# Convertimos a dataframe
research = pd.DataFrame(titulos_abstracts, columns=["Title", "Abstract"])
print("Número de registros: ",len(research))
research.head()

Número de registros:  13300


,Title,Abstract
0,GP-IN: Discovering Pathways into the Geoscienc...,Stony Brook University’s School of Marine and ...
1,Collaborative Research: An Experimental and Mo...,Evaporation is a uniquely important process in...
2,EAGER: Discovery of Next Generation Durable No...,PART 1. NON-TECHNICAL<br/><br/>Colors play a v...
3,RUI: Complex Dynamics in Glass-forming Oxide M...,NON-TECHNICAL DESCRIPTION: Oxide glasses featu...
4,Graduate Research Fellowship Program (GRFP),The National Science Foundation (NSF) Graduate...


# Limpieza de datos

In [25]:
columnas = research.columns
# Reemplazamos los siguientes caracteres por una cadena vacía
research[columnas] = research[columnas].replace({'&lt|br/|&gt|<br/>': ''}, regex=True)
# Eliminamos cualquier carácter que no sea alfanumérico, coma, punto o dos puntos
research[columnas] = research[columnas].replace({'[^0-9a-zA-Z.,: ]': ''}, regex=True)
# Eliminamos duplicados
research.drop_duplicates(subset=["Title"], inplace=True)
research.drop_duplicates(subset=["Abstract"], inplace=True)
# Eliminamos nulos
research.dropna(inplace=True)
# Eliminamos espacios en blanco
research['Title'] = research['Title'].str.strip()
research['Abstract'] = research['Abstract'].str.strip()
# Exportamos el archivo en csv para no reprocesar en el futuro
research.to_csv('research.csv', index=False)

In [29]:
# Importamos los datos
research = pd.read_csv("research.csv")
# Imprimimos el número de registros
print("Número de registros después de limpieza:", len(research))
research_df

Número de registros después de limpieza: 10904


,Title,Abstract
0,Collaborative Research: Excellence in Research...,Head and heart development are closely intertw...
1,Workshop on Replication of a CommunityEngaged ...,The National Academy of Engineering identified...
2,Brazos Analysis Seminar,This award provides three years of funding to ...
3,Collaborative Research: ECR EIE DCL: The Devel...,"This collaborative research project, involving..."
4,Research Initiation Award: Microwave Synthesis...,Research Initiation Awards provide support for...
...,...,...
10899,SaTC: CORE: Small: Mining Strong Security from...,Radio Frequency Identification RFID has played...
10900,National Center for Next Generation Manufacturing,Recent studies have highlighted the nations in...
10901,NSFBSF: Dynamics and Operator Algebras beyond ...,"This project links two mathematical fields, dy..."
10902,Collaborative Research: SaTC: CORE: Medium: Na...,Recent years have seen a dramatic rise in mobi...


# Embeddings

In [30]:
# Importamos el modelo para obtener el embedding del abstract
model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

In [31]:
# Definimos un método de embedding
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
  try:
    embeddings = model.get_embeddings(sentences)
    return [embedding.values for embedding in embeddings]
  except Exception:
      return [None for _ in range(len(sentences))]

In [32]:
# Función generadora para producir lotes de abstracts
def generate_batches(
    abstracts: List[str], batch_size: int
) -> Generator[List[str],None,None]:
    for i in range(0, len(abstracts), batch_size):
      yield abstracts[i:i+batch_size]

def encode_text_toembedding_batched(
    abstracts: List[str], api_calls_per_minute: int = 20, batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Preparamos los batches usando el generador
    batches = generate_batches(abstracts, batch_size)

    seconds_per_job = 60/api_calls_per_minute

    with ThreadPoolExecutor() as executor:
      futures = []
      for batch in tqdm(
          batches, total = math.ceil(len(abstracts) / batch_size), position=0
      ):
          futures.append(
              executor.submit(functools.partial(encode_texts_to_embeddings), batch)
          )
          time.sleep(seconds_per_job)

      for future in futures:
          embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(abstracts, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

In [12]:
# Generamos los embeddings
abstracts = research['Abstract'].values.tolist()
response = encode_text_toembedding_batched(abstracts, api_calls_per_minute=20)

  0%|          | 0/2181 [00:00<?, ?it/s]

In [13]:
# Guardamos los embeddings para no tener que volver a crearlos después
with open('embeddings_response.pkl', 'wb') as archivo:
    pickle.dump(response, archivo)

In [33]:
# Importamos los embeddings
with open ('embeddings_response.pkl','rb') as archivo:
  response = pickle.load(archivo)

In [34]:
# Verificamos que se hayan creado todos los embeddings
print('item 0')
print(f'\ttype:{type(response[0])}')
print(f'\tvalues:{len(response[0])}')
print(f'Todas las solicitudes de embeddings se completaron correctamente: {all(response[0])} ')

item 0
	type:<class 'list'>
	values:10904
Todas las solicitudes de embeddings se completaron correctamente: True 


In [35]:
# Propiedades de los embeddings
print('Item 1')
print(f'\ttype: {type(response[1])}')
print(f'\tshape: {response[1].shape}')
print(f'\tvector data type: {response[1].dtype}')

Item 1
	type: <class 'numpy.ndarray'>
	shape: (10904, 768)
	vector data type: float64


In [2]:
# Imprimimos el embedding del primer abstract solo para ver
#print(f'Original text: \n{research["Abstract"][0]}\nEmbedding vector:')
#print(response[1][0])

# Clusterización

Con los embeddings calculados, podemos usarlos para crear clusters que agrupen productos similares.

Aquí aplicaremos el modelo K-Means a los vectores de embedding.



In [36]:
# Obtenemos los embeddings
embeddings = response[1]
# Aplicamos el modelos a los datos
kmeans = KMeans(init ="k-means++", n_clusters=10, n_init="auto").fit(embeddings)

In [37]:
# Obtenemos los resultados del modelo
predictions = kmeans.predict(embeddings)

In [38]:
# Añadimos una columna al dataframe con los resultados del modelo
research['cluster'] = predictions
research.head()

,Title,Abstract,cluster
0,Collaborative Research: Excellence in Research...,Head and heart development are closely intertw...,9
1,Workshop on Replication of a CommunityEngaged ...,The National Academy of Engineering identified...,3
2,Brazos Analysis Seminar,This award provides three years of funding to ...,0
3,Collaborative Research: ECR EIE DCL: The Devel...,"This collaborative research project, involving...",3
4,Research Initiation Award: Microwave Synthesis...,Research Initiation Awards provide support for...,9


In [39]:
# Contamos el número de abstracts por cluster
research.groupby('cluster').count()['Title']

cluster
0     803
1    1570
2     699
3    1176
4     793
5    1340
6     968
7     992
8    1498
9    1065
Name: Title, dtype: int64

In [40]:
# Guardamos el modleo
model_local_file = 'model.pkl'
with open(model_local_file, 'wb') as model_file:
  pickle.dump(kmeans, model_file)

Por el momento, hasta aquí dejaremos el análisis, pero sin duda se requiere determinar el valor óptimo de clusters.

# Clasificación

Otra aproximación al problema es por medio de la utilización de un modelo de IA generativa de generación de texto, el cual nos permitirá clasificar los papers en diferentes categorías, las cuales le serán proporcionadas al modelo. Si bien, podemos extraer las categorías de los archivos xml mismos, por conveniencia, utilizaremos las áreas del conocimiento mencionadas en esta liga: http://webofscience.help.clarivate.com/en-us/Content/research-areas.html

In [41]:
# Importamos ahora el modelo de generación de texto
model = TextGenerationModel.from_pretrained("text-bison")

In [36]:
# Aplicamos el modelo para clasificar los papers en las diferentes categorías
research['Clasificación'] = research.apply(lambda x:model.predict(
    """Problema de opción múltiple: Define la categoría del ticket.
    Categorías:
    - Arts & Humanities
    - Life Sciences & Biomedicine
    - Physical Sciences
    - Social Sciences
    - Technology""" +
      'Ticket: ' + x['Abstract'] +
      "Categoría:").text, axis=1)

Nota: La ejecución del modelo tomó aproximadamente 3 horas y tuvo un costo aproximado de 300 pesos haha

In [38]:
# Mostramos el dataframe con las categorías
research

,Title,Abstract,category,Clasificación
0,Collaborative Research: Excellence in Research...,Head and heart development are closely intertw...,3,Ciencias de la vida y biomedicina
1,Workshop on Replication of a CommunityEngaged ...,The National Academy of Engineering identified...,9,Social Sciences
2,Brazos Analysis Seminar,This award provides three years of funding to ...,4,Categoría: Ciencias Físicas
3,Collaborative Research: ECR EIE DCL: The Devel...,"This collaborative research project, involving...",9,Social Sciences
4,Research Initiation Award: Microwave Synthesis...,Research Initiation Awards provide support for...,8,Ciencias de la vida y biomedicina
...,...,...,...,...
10899,SaTC: CORE: Small: Mining Strong Security from...,Radio Frequency Identification RFID has played...,1,Tecnología
10900,National Center for Next Generation Manufacturing,Recent studies have highlighted the nations in...,9,Tecnología
10901,NSFBSF: Dynamics and Operator Algebras beyond ...,"This project links two mathematical fields, dy...",2,Physical Sciences
10902,Collaborative Research: SaTC: CORE: Medium: Na...,Recent years have seen a dramatic rise in mobi...,1,Ciencias Sociales


In [ ]:
# Exportamos el dataframe
research.to_csv('research_classification.csv', index=False)